In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings
import os
import time
#import tensorflow_datasets as tfds
warnings.filterwarnings("ignore")

 ## Import Data

In [2]:
#data, info = tfds.load("mnist", with_info=True)

In [3]:
#train_data = data['train']
#ds = train_data.take(5)
data = pd.read_csv('../../data/UCr/abundance.tsv', index_col=0, sep='\t', header=None)
to_drop = data.loc[(data < 0.1).all(axis=1)]
data = data.drop(to_drop.index)
labels = np.genfromtxt('../../data/UCr/labels.txt', dtype=np.str_, delimiter=',')

data_array = np.array(data).T

class_0 = data_array[labels.astype(int) == 0]
class_1 = data_array[labels.astype(int) == 1]

train_data = class_1
train_data.shape

(44, 22)

In [4]:
# for example in train_data:
#     print(list(example.keys()))
#     image = example["image"]
#     label = example["label"]
#     plt.imshow(image)
#     plt.show()
#     print(image.shape)
#     break

# Training Using Fit

## Model

In [5]:
class Generator(tf.keras.Model):
    def __init__(self):
        super(Generator, self).__init__()

        self.dense_1 = tf.keras.layers.Dense(300, activation=tf.nn.relu)
        self.dense_2 = tf.keras.layers.Dense(300, activation=tf.nn.relu)
        self.dense_3 = tf.keras.layers.Dense(300, activation=tf.nn.relu)
        self.dense_4 = tf.keras.layers.Dense(22, activation=tf.nn.relu)
        #self.reshape = tf.keras.layers.Reshape((28, 28, 1))

    def call(self, inputs):
        x = self.dense_1(inputs)
        x = self.dense_2(x)
        x = self.dense_3(x)
        x = self.dense_4(x)
        return x


class Discriminator(tf.keras.Model):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.flatten = tf.keras.layers.Flatten()
        self.dense_1 = tf.keras.layers.Dense(300, activation=tf.nn.leaky_relu )
        self.dense_2 = tf.keras.layers.Dense(300, activation=tf.nn.leaky_relu )
        self.dense_3 = tf.keras.layers.Dense(300, activation=tf.nn.leaky_relu)
        self.dense_4 = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.dense_1(x)
        x = self.dense_2(x)
        x = self.dense_3(x)
        return self.dense_4(x)

## Loss Functions

In [6]:
# loss object: Binary Crossentropy Loss
# real_output: Image from Data
# fae_output: Image from Generator
def discriminator_loss(loss_object, real_output, fake_output):
    real_loss = loss_object(tf.ones_like(real_output), real_output)
    fake_loss = loss_object(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

# loss object: Binary Crossentropy Loss
# discriminator_probability: Result from Discriminator 0 = Fake 1 = Real
def generator_loss(loss_object, discriminator_probability):
    return loss_object(tf.ones_like(discriminator_probability), discriminator_probability)

# Normalize Image between [-1,1]
def normalize(x):
    #image = tf.cast(x['image'], tf.float32)
    #image = (image / 127.5) - 1
    return image

# Save Image sample from Generator
def save_imgs(epoch, generator, noise):
    gen_imgs = generator(noise)

    fig = plt.figure(figsize=(4, 4))

    for i in range(gen_imgs.shape[0]):
        plt.subplot(4, 4, i + 1)
        plt.imshow(gen_imgs[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')

    fig.savefig("images/mnist_%d.png" % epoch)

### Callback: They allow us to save image during training

In [7]:
class Training_Callback(tf.keras.callbacks.Callback):
    def __init__(self, latent_dim, saving_rate):
        super(Training_Callback, self).__init__()
        self.latent_dim = latent_dim
        self.saving_rate = saving_rate
        
    # Save Image sample from Generator
#     def save_imgs(self, epoch):
#         # Number of images = 16
#         seed = tf.random.normal([16, self.latent_dim])
#         gen_imgs = self.model.generator(seed)
        
#         fig = plt.figure(figsize=(4, 4))

#         for i in range(gen_imgs.shape[0]):
#             plt.subplot(4, 4, i + 1)
#             plt.imshow(gen_imgs[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
#             plt.axis('off')

#         fig.savefig("images/mnist_%d.png" % epoch)
    
    # Called after each epoch
#     def on_epoch_end(self, epoch, logs=None):
#         # Save image after 50 epochs
#         if epoch % 50 == 0:
#             self.save_imgs(epoch)
            
#         if epoch > 0 and epoch % self.saving_rate == 0:
#             save_dir = "./models/model_epoch_" + str(epoch)
#             if not os.path.exists(save_dir):
#                 os.makedirs(save_dir)
#             self.model.discriminator.save_weights(save_dir + '/discriminator_%d' % epoch)
#             self.model.generator.save_weights(save_dir + '/generator_%d' % epoch)
            
#         self.best_weights = self.model.get_weights()

## Modify train_step function

In [8]:
class GAN(tf.keras.Model):
    # define the models
    def __init__(self, discriminator, generator, latent_dim):
        super(GAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
    
    # Define the compiler
    def compile(self, disc_optimizer, gen_optimizer, loss_fn, generator_loss, discriminator_loss):
        super(GAN, self).compile()
        self.disc_optimizer = disc_optimizer
        self.gen_optimizer = gen_optimizer
        self.generator_loss = generator_loss
        self.discriminator_loss = discriminator_loss
        self.loss_fn = loss_fn

        
    # @tf.function: The below function is completely Tensor Code
    # Good for optimization
    @tf.function
    # Modify Train step for GAN
    def train_step(self, images):
        batch_size = tf.shape(images)[0]
        noise = tf.random.normal([batch_size, self.latent_dim])

        # Define the loss function
        with tf.GradientTape(persistent=True) as tape:
            generated_images = self.generator(noise)
            real_output = self.discriminator(images)
            fake_output = self.discriminator(generated_images)
            
            gen_loss = self.generator_loss(self.loss_fn, fake_output)
            disc_loss = self.discriminator_loss(self.loss_fn, real_output, fake_output)

        # Calculate Gradient
        grad_disc = tape.gradient(disc_loss, self.discriminator.trainable_variables)
        grad_gen = tape.gradient(gen_loss, self.generator.trainable_variables)

        # Optimization Step: Update Weights & Learning Rate
        self.disc_optimizer.apply_gradients(zip(grad_disc, self.discriminator.trainable_variables))
        self.gen_optimizer.apply_gradients(zip(grad_gen, self.generator.trainable_variables))
        
        return {"Gen Loss ": gen_loss,"Disc Loss" : disc_loss}

### Set Hyperparameters

In [9]:
latent_dim = 100
epochs = 2000
# Reduce Batch Size if GPU memory overflow
# Better Use Google Colab
batch_size = 2000
# Save model after every
saving_rate = 100

# Random Seed for Shuffling Data
buffer_size = 5000

# defining optimizer for Models
gen_optimizer = tf.keras.optimizers.Adam(0.0001)
disc_optimizer = tf.keras.optimizers.Adam(0.0001)

# Shuffle & Batch Data
#train_dataset = train_data.map(normalize).shuffle(buffer_size , reshuffle_each_iteration=True).batch(batch_size)
train_dataset = train_data

# Define Loss Function
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

### Create GAN & Compiler

In [10]:
disc = Discriminator()
gen = Generator()

Metal device set to: Apple M1 Pro


2023-01-10 02:12:23.633411: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-10 02:12:23.633538: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [11]:
gan = GAN(discriminator=disc, generator=gen, latent_dim=latent_dim)
gan.compile(
    disc_optimizer=disc_optimizer,
    gen_optimizer=gen_optimizer,
    loss_fn=cross_entropy,
    generator_loss = generator_loss,
    discriminator_loss = discriminator_loss
)

### Train Data

In [12]:
training_callback = Training_Callback(10, saving_rate)
gan.fit(
    train_dataset, 
    epochs=epochs,
    callbacks=[training_callback]
)

Epoch 1/2000


2023-01-10 02:12:23.794834: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-01-10 02:12:24.453460: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 1s 227ms/step - Gen Loss : 0.6921 - Disc Loss: 1.3871
Epoch 2/2000
2/2 [==============================] - 0s 24ms/step - Gen Loss : 0.7002 - Disc Loss: 1.3794
Epoch 3/2000
2/2 [==============================] - 0s 19ms/step - Gen Loss : 0.7066 - Disc Loss: 1.3745
Epoch 4/2000
2/2 [==============================] - 0s 17ms/step - Gen Loss : 0.7138 - Disc Loss: 1.3694
Epoch 5/2000
2/2 [==============================] - 0s 15ms/step - Gen Loss : 0.7221 - Disc Loss: 1.3617
Epoch 6/2000
2/2 [==============================] - 0s 15ms/step - Gen Loss : 0.7313 - Disc Loss: 1.3539
Epoch 7/2000
2/2 [==============================] - 0s 49ms/step - Gen Loss : 0.7398 - Disc Loss: 1.3465
Epoch 8/2000
2/2 [==============================] - 0s 19ms/step - Gen Loss : 0.7458 - Disc Loss: 1.3419
Epoch 9/2000
2/2 [==============================] - 0s 16ms/step - Gen Loss : 0.7472 - Disc Loss: 1.3442
Epoch 10/2000
2/2 [==============================] - 0s 19ms/step -

## Save Best Gen & Disc

In [13]:
training_callback.model.generator

In [14]:
disc.save_weights('./models/discriminator')
gen.save_weights('./models/generator')

## Load Weight

In [15]:
from tensorflow.keras.models import load_model
disc1 = Discriminator()
gen1 = Generator()
disc1.load_weights('./models/discriminator')
gen1.load_weights('./models/generator')

In [16]:
noise = tf.random.normal([100, latent_dim])
generated_images = gen1.predict(noise)
generated_images.shape

4/4 [==============================] - 0s 8ms/step


2023-01-10 02:13:30.303435: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


(100, 22)

In [17]:
generated = generated_images.T
generated_df = pd.DataFrame(generated)
#generated_df = generated_df.where(generated_df > 0, 0)
generated_df = generated_df.div(generated_df.sum(axis=0), axis=1)
generated_df

#generated.sum(axis = 0)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.061749,0.089344,0.000000,0.065912,0.000000,0.000000,0.098490,0.096584,0.132466,0.071172,...,0.221957,0.079026,0.161729,0.098227,0.119655,0.085039,0.065221,0.084467,0.078221,0.012609
1,0.000000,0.000000,0.000000,0.004119,0.000000,0.000000,0.000000,0.010135,0.030853,0.000000,...,0.001458,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010612,0.000000,0.000000,...,0.056844,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041757,0.026563,0.000000
4,0.050786,0.009879,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.137283,...,0.000000,0.039636,0.000000,0.004819,0.000000,0.000000,0.000000,0.000000,0.043233,0.000000
5,0.000000,0.037766,0.017963,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.026201,0.042925,0.007689,0.074165,0.023786,0.000000,0.051717,0.039252,0.000000,0.011428,...,0.000000,0.056271,0.070544,0.000000,0.041936,0.000000,0.000000,0.013393,0.026049,0.067006
7,0.088804,0.000000,0.082477,0.046138,0.046181,0.176695,0.060486,0.063859,0.043125,0.002711,...,0.069594,0.030417,0.039625,0.137213,0.004993,0.040767,0.149288,0.072742,0.000000,0.021774
8,0.000000,0.021383,0.057641,0.000000,0.000000,0.055092,0.000000,0.001139,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.033665,0.000000,0.039101,0.000000,0.000000,0.001460,0.081432
9,0.104021,0.096766,0.137691,0.162455,0.152537,0.250400,0.142568,0.163105,0.214446,0.168471,...,0.220066,0.105465,0.136214,0.230856,0.076558,0.127196,0.092268,0.069173,0.139009,0.115531


In [18]:
generated.sum(axis = 0)
#len(generated[0])

array([0.9210829 , 0.93923616, 1.1224692 , 0.7850607 , 0.87120914,
       0.71964663, 0.7515457 , 0.7114385 , 0.605774  , 0.694028  ,
       0.7247442 , 0.6641045 , 0.6832963 , 1.1234432 , 0.84900314,
       0.5018444 , 0.8955259 , 0.7565912 , 0.74087   , 1.24983   ,
       0.5918908 , 1.0249969 , 0.8551552 , 0.55330193, 0.61034983,
       0.97806317, 0.62989926, 1.0959988 , 0.6359982 , 0.7595498 ,
       0.99791   , 1.0447646 , 0.91080475, 0.72093   , 0.8698319 ,
       0.7108345 , 0.8239416 , 0.90745986, 0.7411122 , 0.77813846,
       0.70484865, 0.97287285, 0.70060796, 0.6827772 , 0.6093881 ,
       0.8387405 , 0.60781145, 0.6451643 , 0.7211541 , 0.85710365,
       0.86068714, 0.6917498 , 0.8044762 , 0.6411984 , 0.813892  ,
       0.680231  , 0.9968618 , 1.012197  , 1.3687525 , 0.7242125 ,
       0.941016  , 0.62493557, 1.1277984 , 0.918177  , 0.8227136 ,
       0.9038913 , 0.840852  , 0.97975135, 1.0046259 , 0.8326146 ,
       0.81217915, 0.94553715, 0.66857266, 0.73195577, 0.83403

In [19]:
generated_df.to_csv('1.csv', sep=',', index=True, header=False)